Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cobra

from yeast8model import Yeast8Model

# Construct model

In [ ]:
from yeast8model import biomass_component_list_orig

glucose_bounds = (-4.75, 0) # gives a sensible growth rate for wt
wt_y8 = Yeast8Model("./models/yeast-GEM_8-6-0.xml", growth_id="r_2111", biomass_id="r_4041")
wt_y8.biomass_component_list = biomass_component_list_orig
wt_y8.model.reactions.r_1714.bounds = glucose_bounds
wt_y8.add_media_components(['r_1992'])

sol = wt_y8.optimize()

In [ ]:
from yeast8model import biomass_component_list_orig

wt_y8 = Yeast8Model("./models/yeast-GEM_8-6-0.xml")
glucose_bounds = (-4.75, 0) # gives a sensible growth rate for wt
wt_y8.add_media_components(['r_1992'])
wt_y8.model.reactions.r_1714.bounds = glucose_bounds
wt_y8.biomass_component_list = biomass_component_list_orig

In [ ]:
sol_orig = wt_y8.optimize()

In [ ]:
sol_orig

In [ ]:
sol_orig.fluxes["r_2111"]

In [ ]:
abl_orig = wt_y8.ablate()

In [ ]:
abl_orig

In [ ]:
fig, ax = plt.subplots()
wt_y8.ablation_barplot(ax, abl_orig)
plt.show()

# Effect of flux penalty on ablation

Import more dependencies and define new objects

(will probably move this from notebook to script/module)

In [ ]:
import time

Main work...

In [ ]:
penalty_coefficient = 0.0
wt_y8.set_flux_penalty(penalty_coefficient=penalty_coefficient)

In [ ]:
sol_pen = wt_y8.optimize()

In [ ]:
sol_pen.fluxes

In [ ]:
sol_pen.fluxes["r_2111"]

In [ ]:
wt_y8.model.objective

In [ ]:
wt_y8._penalty_coefficient

In [ ]:
diff = sol_pen.fluxes - sol_orig.fluxes
print(diff.min())
print(diff.max())

In [ ]:
abl_res = wt_y8.ablate()

In [ ]:
abl_res

Define range of penalty coefficients to study:

In [ ]:
penalty_coeff_range = np.linspace(0, 1, 3)
penalty_coeff_range

In [ ]:
# Define data structure to store investigation results.

# Alternative considered: dict with floats as keys.
# Shot that down because I think that's weird and not the point of a dict,
# especially if the penalty_coeff_range can change.
effect_list = [{'penalty_coeff': penalty_coeff, 'ablation_result': None} for penalty_coeff in penalty_coeff_range]

In [ ]:
for effect_item in effect_list:
    print(effect_item)
    
    print(f"coeff {effect_item['penalty_coeff']}")
    start = time.time()
    wt_y8.set_flux_penalty(penalty_coefficient=effect_item["penalty_coeff"])
    end = time.time()
    print(f"penalty set with coeff.  elapsed time: {end - start} s")
    
    sol = wt_y8.optimize() # check if this line is necessary
    
    start = time.time()
    ablation_result = wt_y8.ablate(verbose=False)
    end = time.time()
    print(f"ablated.  elapsed time: {end - start} s")
    print(ablation_result)
          
    effect_item["ablation_result"] = ablation_result

In [ ]:
effect_list